# Systems Programming

## Lecture 12: External Libraries

### Amir Atapour-Abarghouei

amir.atapour-abarghouei@durham.ac.uk


# Recap - Makefiles

- When we have a number of files to compile, we need a rule-set.

- The `make` command provides this.

- Requires a rule-file called the `Makefile`.

- Declarative programming style set of rules for building the program.
```
    target [target ...]: [component ...]
           [command 1]
               ...
           [command n]
```
- `target` - what you want to make
- `component` - something which needs to exist (might need another rule)

# Recap - Makefiles: Macros

- Macros can be used to store definitions, e.g. `AUTHOR = Amir Atapour`
    
- They can be generated from commands, e.g. `DATE = ‘date‘`
    
- And used in the `Makefile`

```
all:
    echo $(AUTHOR) compiled this on $(DATE)
```

- running this gives:

```
Amir Atapour compiled this on Mon 12 Nov 12:34:36 GMT 2021
```


## Today

# External Libraries

# External Libraries

- One of the reasons why C is so popular is the huge collection of tried and tested libraries available across many different computing platforms, e.g. `OpenGL`:

<center><img src="images/openGL.png" alt="openGL" width="600"></center>

- Commands from your program are sent by the API to the graphics hardware which generates pixels for display.

*OpenGL behaves like a state machine.

# OpenGL Programming

On its own OpenGL is:

1. Low level
2. OS independent

Hence it is usually used with:

- GLU a utility library with high level shape support
- GLUT utility library for window creation and I/O

# Commonly-Used C Libraries

- general: `libglib` / `libgobject` / `libpthread`
- console: `libncurses`
- 2D graphics: `libX11` / `libSDL`
- 3D graphics: `libGL` / `libGLU` / `libGLUT`
- GUI toolkits: `libgtk` / `libQT`
- Images: `libjpeg` / `libpng` / `libgif`

# Commonly-Used C Libraries

- text rendering: `libpango` / `libfreetype`
- sound: `libasound` / `libSDL`
- compression: `libz` / `libgzip` / `libbz2`
- encryption: `libcrypt` / `libssl` / `libgssapi` / `libkrb5`
- XML: `libxml2`
- web: `libcurl`

# Compilation Model

<center><img src="images/compile.png" alt="compilation model" width="800"></center>

# Usage of Libraries

- If a library is *statically linked*, then a copy of the library is included in the executable.

- C/C++/assembly can be combined.

- Often bound to other languages e.g. php, XML, curl.

- Many of these libraries will be *dynamically linked*.

- Try `ldd /usr/bin/php` on Linux to list dynamic dependencies

- LGPL (Lesser Gnu Public License) often used.

# Dynamic vs Static Linking

- Dynamic linking takes place at run-time not build-time.

    - Reduces filespace demands (bloat) by keeping only one copy of the library.
    - Only one copy of the library is loaded into memory.
    - Can help with updates e.g. for security.
    - Using dynamic linking can be slightly slower than static linking.

# Dynamic vs Static Linking

- Dynamic libraries are called differently by OSs.

    - Linux: shared objects (`.so`)

    - Windows: Dynamic Link Libraries (`.dll`)

    - OSX: `.dylib`

- Can lead to “*DLL Hell*”: many versions of the same dynamic library
    - Best to include version number with library.

# Side Note: Semantic Versioning

`Python 3.9.6`

- `3`: Major version
- `9`: Minor version
- `6`: Patch version

If any change to the software is completely backwards compatible, only the **Patch Version** is incremented.
- Nothing is added or removed (externally, it is as if nothing is changed).
    - e.g. security fixes

# Side Note: Semantic Versioning

`Python 3.9.6`

- `3`: Major version
- `9`: Minor version
- `6`: Patch version

If some functionality is added to the library (or software) and nothing is removed, then the **Minor Version** is incremented and the Patch Version is set to zero.
- Older software will work with the new version.


# Side Note: Semantic Versioning

`Python 3.9.6`

- `3`: Major version
- `9`: Minor version
- `6`: Patch version

If backwards incompatible changes are made to the library, the **Major Version** is incremented and Minor and Patch are set to zero.
- e.g. if a function is removed or renamed.

# Creating a Static Library

- A static library is effectively just an archive containing object (`.o`) files and is created with the archiver `ar`.

- In UNIX, static libraries use the `.a` extension.

```
gcc -c linkedlist.c -o bin/static/linkedlist.o
gcc -c anotherfile.c -o bin/static/anotherfile.o
ar rcs bin/static/libLL.a bin/static/linkedlist.o \
       bin/static/anotherfile.o
```

- `r` means that if the library already exists, replace it. `c` means create the library if it did not exist. `s` is to create a sorted index of the library.

# Creating a Static Library

 To link statically:
 
- Use the `-L` flag to list a (non-standard) directory where the library can be found.

- Use the `-l` flag to give the name of the library.
    - Note that it assumes the library starts with lib and has the extension `.a` (static) or `.so` (dynamic)
```
gcc  main.o  -Lbin/static -lLL -o bin/main-static
```
Can now run: `bin/main-static`

# Example

## foo.h

```c
extern void foo1(void);
extern void foo2(void);

```

# Example

## foo1.c

```c
#include <stdio.h>
 
 
void foo1(void){
    printf("and this is part of the static library - 1\n");
}

```

# Example

## foo2.c

```c
#include <stdio.h>
 
 
void foo2(void){
    printf("and this is part of the static library - 2\n");
}

```


## main.c

```c
#include <stdio.h>
#include "foo.h"
 
int main(void){
    printf("This is a static library test...\n");
    foo1();
    foo2();
    return 0;
}

```

# Let's create a static library...

# Creating a Shared Library - *dynamically linked*

- Objects files for a shared library need to be compiled with the `-fPIC` option.
- On UNIX, shared libraries use the `.so` extension.
- PIC=“Position Independent Code”, since we don’t know where in memory the library will be loaded at run-time.

```
gcc -fPIC -c linkedlist.c -o bin/dynamic/linkedlist.o
gcc -fPIC -c anotherfile.c -o bin/dynamic/anotherfile.o
gcc -shared bin/dynamic/linkedlist.o \
    bin/dynamic/anotherfile.o -o bin/dynamic/libLL.so
```

- To link dynamically:
    - `gcc main.o -Lbin/dynamic -lLL -o bin/main-dynamic`

# Creating a Shared Library

- If we try to run it, we get an error:

```
bin/main-dynamic: error while loading shared libraries:
    libLL.so: cannot open shared object file: No such
    file or directory
```

- We need to tell the operating system where to find the library.

# Creating a Shared Library

We need to tell the operating system where to find the library:

```
# In bash:
export \
LD_LIBRARY_PATH=`pwd`/bin/dynamic/:$LD_LIBRARY_PATH
# In tcsh (the default shell on mira):
setenv LD_LIBRARY_PATH `pwd`/bin/dynamic:$LD_LIBRARY_PATH
```

- (Note that ` above is a backtick)

# Example

## foo.h

```c
#ifndef foo_h__

    #define foo_h__
    extern void foo(void);
 
#endif  // foo_h__
```

# Example

## foo.c

```c
#include <stdio.h>
 
void foo(void){
    printf("This is a shared library");
}
```

# Example

## main.c

```c
#include <stdio.h>
#include "foo.h"
 
int main(void){
    printf("This is a shared library test...");
    foo();
    return 0;
}
```

# Let's create a shared library...

# Summary

- External libraries

- Static libraries

- Dynamic libraries

- How libraries are created